In [146]:
import jieba
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [147]:
from os import listdir
from os.path import isfile, join,splitext
datapath = 'Stage_2\dataPublicComplete_s2\dataPublicComplete'
txt_fnames = [splitext(f)[0] for f in listdir(datapath) if isfile(join(datapath, f))]

In [148]:
crop_list = open('Keywords/02crop.list.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()

pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()

In [149]:
from itertools import chain
vector_dict = {}
for idx,line in enumerate(chain(crop_line_sep,pest_line_sep,chem_line_sep)):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[word] = idx

In [150]:
vectors = {}
for fname in txt_fnames:
    txt = open(datapath+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    vectors[fname] = [0]*764
    for seg in seg_list:
        if(seg in vector_dict):
            vectors[fname][vector_dict[seg]] += 1

In [151]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2*764, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 20),
            nn.Sigmoid(),
            nn.Linear(20, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [152]:
net = NeuralNetwork()
net.load_state_dict(torch.load('best_checkpoint_last.pth'))
net.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

In [153]:
out = []
with torch.no_grad():
    for fname1 in txt_fnames:
        for fname2 in txt_fnames:
            if(fname1 == fname2):continue
            else:
                comb_vec = torch.tensor(vectors[fname1]+vectors[fname2])
                comb_vec = comb_vec.type(torch.FloatTensor).to(device)
                lbl = net(comb_vec)
                out.append([(fname1,fname2),lbl.cpu()])


In [161]:
better = []
for o in out:
    if(o[1][1] > 0 ):
        better.append(o[0])

In [162]:
print(len(better))

6298


In [122]:
for o in out:
    if(o[0] == ('1032','1036')):
        print(o)
better.sort(key=lambda x:int(x[0]))

[('1032', '1036'), tensor([ 2.5751, -2.8332])]


In [163]:
import csv
with open('val.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Test"]+["Reference"])
    for row in better:  
        spamwriter.writerow(row)

[[('1001', '1004'), tensor([ 2.8223, -3.3220])],
 [('1001', '1006'), tensor([ 2.9246, -3.4106])],
 [('1001', '1008'), tensor([ 2.7173, -3.2599])],
 [('1001', '1009'), tensor([ 2.3772, -2.9819])],
 [('1001', '1012'), tensor([ 2.7990, -3.3099])],
 [('1001', '1014'), tensor([ 2.3454, -2.9554])],
 [('1001', '1019'), tensor([-1.2686,  0.1840])],
 [('1001', '102'), tensor([-0.2560, -0.7116])],
 [('1001', '1022'), tensor([-0.9860, -0.0671])],
 [('1001', '1024'), tensor([ 0.8966, -1.7245])],
 [('1001', '1027'), tensor([ 3.0371, -3.4940])],
 [('1001', '1028'), tensor([ 2.8781, -3.3751])],
 [('1001', '1029'), tensor([ 2.8928, -3.3870])],
 [('1001', '1032'), tensor([ 2.9018, -3.3942])],
 [('1001', '1034'), tensor([ 0.1076, -1.0337])],
 [('1001', '1036'), tensor([ 2.9018, -3.3942])],
 [('1001', '1038'), tensor([ 3.0023, -3.4662])],
 [('1001', '1040'), tensor([-0.0194, -0.9219])],
 [('1001', '1048'), tensor([ 2.8739, -3.3737])],
 [('1001', '1050'), tensor([ 2.3050, -2.9244])],
 [('1001', '1051'), t

In [155]:
out.sort(key = lambda x:x[1][1],reverse = True)

In [160]:
out[7000:7100]

[[('1156', '1209'), tensor([-0.9283, -0.1058])],
 [('87', '24'), tensor([-0.9380, -0.1059])],
 [('96', '24'), tensor([-0.9380, -0.1059])],
 [('435', '409'), tensor([-0.9487, -0.1060])],
 [('159', '269'), tensor([-0.9522, -0.1061])],
 [('798', '754'), tensor([-0.9283, -0.1063])],
 [('24', '464'), tensor([-0.9605, -0.1066])],
 [('473', '71'), tensor([-0.9309, -0.1067])],
 [('910', '714'), tensor([-0.9716, -0.1068])],
 [('742', '1336'), tensor([-0.9382, -0.1069])],
 [('697', '748'), tensor([-0.9421, -0.1071])],
 [('930', '79'), tensor([-0.9388, -0.1079])],
 [('79', '714'), tensor([-0.9634, -0.1090])],
 [('134', '13'), tensor([-0.9147, -0.1091])],
 [('490', '427'), tensor([-0.9226, -0.1093])],
 [('202', '488'), tensor([-0.9583, -0.1096])],
 [('742', '1345'), tensor([-0.9341, -0.1097])],
 [('161', '1384'), tensor([-0.9469, -0.1100])],
 [('941', '734'), tensor([-0.9336, -0.1100])],
 [('669', '459'), tensor([-0.9219, -0.1101])],
 [('74', '45'), tensor([-0.9450, -0.1103])],
 [('854', '1368'), 